In [1]:
import os
import sys
import json
import glob
import torch
import torch.nn as nn
import torch.nn.functional as F
from reformer_pytorch import Reformer

In [2]:
base_dir = os.path.dirname(os.getcwd())
data_dir = os.path.join(base_dir, "data", "original")
train_files = glob.glob(os.path.join(data_dir, "train", "*", "*.obj"))
valid_files = glob.glob(os.path.join(data_dir, "val", "*", "*.obj"))
print(len(train_files), len(valid_files))

src_dir = os.path.join(base_dir, "src")
sys.path.append(os.path.join(src_dir))

7003 1088


In [3]:
from utils import load_pipeline
from tokenizers import DecodeVertexTokenizer, FaceTokenizer

In [4]:
v_batch, f_batch = [], []
for i in range(3):
    v, f = load_pipeline(train_files[i])
    v = torch.tensor(v)
    f = torch.tensor(f)
    v_batch.append(v)
    f_batch.append(f)
    print(v.shape, f.shape)
    print("="*60)

torch.Size([655, 3]) torch.Size([1317, 3])
torch.Size([310, 3]) torch.Size([676, 3])
torch.Size([396, 3]) torch.Size([1184, 3])


In [5]:
dec_tokenizer = DecodeVertexTokenizer()

In [6]:
input_tokens = dec_tokenizer.tokenize(v_batch)
input_tokens

{'value_tokens': tensor([[  0, 163, 139,  ..., 125,   1,   2],
         [  0, 137, 164,  ...,   2,   2,   2],
         [  0, 165, 101,  ...,   2,   2,   2]]),
 'target_tokens': tensor([[163, 139, 135,  ...,   1,   2,   2],
         [137, 164, 132,  ...,   2,   2,   2],
         [165, 101, 136,  ...,   2,   2,   2]]),
 'coord_type_tokens': tensor([[0, 1, 2,  ..., 3, 0, 0],
         [0, 1, 2,  ..., 0, 0, 0],
         [0, 1, 2,  ..., 0, 0, 0]]),
 'position_tokens': tensor([[  0,   1,   1,  ..., 655,   0,   0],
         [  0,   1,   1,  ...,   0,   0,   0],
         [  0,   1,   1,  ...,   0,   0,   0]]),
 'padding_mask': tensor([[False, False, False,  ..., False, False,  True],
         [False, False, False,  ...,  True,  True,  True],
         [False, False, False,  ...,  True,  True,  True]])}

In [7]:
class VertexEmbedding(nn.Module):
    
    def __init__(self, 
                 vocab_value=256, pad_idx_value=2, 
                 vocab_coord_type=4, pad_idx_coord_type=0,
                 vocab_position=1000, embed_dim=256):
        
        super().__init__()
        
        self.value_embed = nn.Embedding(
            vocab_value, embed_dim, padding_idx=pad_idx_value
        )
        self.coord_type_embed = nn.Embedding(
            vocab_coord_type, embed_dim, padding_idx=pad_idx_coord_type
        )
        self.position_embed = nn.Embedding(
            vocab_position, embed_dim
        )
        
    def forward(self, value_tokens, coord_type_tokens, position_tokens):
        embed = self.value_embed(value_tokens)
        embed = embed + self.coord_type_embed(coord_type_tokens)
        embed = embed + self.position_embed(position_tokens)
        return embed

In [8]:
embed = VertexEmbedding(embed_dim=128)

In [9]:
input_tokens

{'value_tokens': tensor([[  0, 163, 139,  ..., 125,   1,   2],
         [  0, 137, 164,  ...,   2,   2,   2],
         [  0, 165, 101,  ...,   2,   2,   2]]),
 'target_tokens': tensor([[163, 139, 135,  ...,   1,   2,   2],
         [137, 164, 132,  ...,   2,   2,   2],
         [165, 101, 136,  ...,   2,   2,   2]]),
 'coord_type_tokens': tensor([[0, 1, 2,  ..., 3, 0, 0],
         [0, 1, 2,  ..., 0, 0, 0],
         [0, 1, 2,  ..., 0, 0, 0]]),
 'position_tokens': tensor([[  0,   1,   1,  ..., 655,   0,   0],
         [  0,   1,   1,  ...,   0,   0,   0],
         [  0,   1,   1,  ...,   0,   0,   0]]),
 'padding_mask': tensor([[False, False, False,  ..., False, False,  True],
         [False, False, False,  ...,  True,  True,  True],
         [False, False, False,  ...,  True,  True,  True]])}

In [10]:
print(
    input_tokens["value_tokens"].shape,
    input_tokens["coord_type_tokens"].shape,
    input_tokens["position_tokens"].shape
)

torch.Size([3, 1968]) torch.Size([3, 1968]) torch.Size([3, 1968])


In [11]:
emb = embed(
    input_tokens["value_tokens"], 
    input_tokens["coord_type_tokens"], 
    input_tokens["position_tokens"]
)
emb.shape

torch.Size([3, 1968, 128])

In [12]:
reformer = Reformer(dim=128, depth=2, max_seq_len=8192, bucket_size=24)

In [13]:
output = reformer(emb)
output.shape

torch.Size([3, 1968, 128])

In [14]:
class Config(object):
    
    def write_to_json(self, out_path):
        with open(out_path, "w") as fw:
            json.dump(self.config, fw, indent=4)
            
    def __getitem__(self, key):
        return self.config[key]

In [105]:
class VertexPolyGenConfig(Config):
    
    def __init__(self,
                 embed_dim=256, 
                 max_seq_len=2400, 
                 tokenizer__bos_id=0,
                 tokenizer__eos_id=1,
                 tokenizer__pad_id=2,
                 embedding__vocab_value=256 + 3, 
                 embedding__vocab_coord_type=4, 
                 embedding__vocab_position=1000, 
                 embedding__pad_idx_coord_type=0,
                 embedding__pad_idx_value=2,
                 reformer__depth=12,
                 reformer__heads=8,
                 reformer__n_hashes=8,
                 reformer__bucket_size=48,
                 reformer__causal=True,
                 reformer__lsh_dropout=0.2, 
                 reformer__ff_dropout=0.2,
                 reformer__post_attn_dropout=0.2,
                 reformer__ff_mult=4):
        
        # tokenizer config
        tokenizer_config = {
            "bos_id": tokenizer__bos_id,
            "eos_id": tokenizer__eos_id,
            "pad_id": tokenizer__pad_id,
            "max_seq_len": max_seq_len,
        }
        
        # embedding config
        embedding_config = {
            "vocab_value": embedding__vocab_value,
            "vocab_coord_type": embedding__vocab_coord_type,
            "vocab_position": embedding__vocab_position,
            "pad_idx_value": embedding__pad_idx_value,
            "pad_idx_coord_type": embedding__pad_idx_coord_type,
            "embed_dim": embed_dim,
        }
        
        # reformer info
        reformer_config = {
            "dim": embed_dim,
            "depth": reformer__depth,
            "max_seq_len": max_seq_len,
            "heads": reformer__heads,
            "bucket_size": reformer__bucket_size,
            "n_hashes": reformer__n_hashes,
            "causal": reformer__causal,
            "lsh_dropout": reformer__lsh_dropout, 
            "ff_dropout": reformer__ff_dropout,
            "post_attn_dropout": reformer__post_attn_dropout,
            "ff_mult": reformer__ff_mult,
        }
        
        self.config = {
            "embed_dim": embed_dim,
            "max_seq_len": max_seq_len,
            "tokenizer": tokenizer_config,
            "embedding": embedding_config,
            "reformer": reformer_config,
        }

In [124]:
class VertexPolyGen(nn.Module):
    
    def __init__(self, model_config):
        super().__init__()
        self.tokenizer = DecodeVertexTokenizer(**model_config["tokenizer"])
        self.embedding = VertexEmbedding(**model_config["embedding"])
        self.reformer = Reformer(**model_config["reformer"])
        self.layernorm = nn.LayerNorm(model_config["embed_dim"])
        self.loss_func = nn.CrossEntropyLoss(ignore_index=model_config["tokenizer"]["pad_id"])
    
    def forward(self, tokens, device=None):
        
        hs = self.embedding(
            tokens["value_tokens"], 
            tokens["coord_type_tokens"], 
            tokens["position_tokens"]
        )
        
        hs = self.reformer(
            hs, input_mask=tokens["padding_mask"]
        )
        hs = self.layernorm(hs)
        
        return hs
        
    def __call__(self, inputs, device=None):
        tokens = self.tokenizer.tokenize(inputs)
        tokens = {k: v.to(device) for k, v in tokens.items()}
        
        hs = self.forward(tokens, device=device)
        hs = F.linear(hs, self.embedding.value_embed.weight)
        BATCH, LENGTH, EMBED = hs.shape
        hs = hs.reshape(BATCH*LENGTH, EMBED)
        targets = tokens["target_tokens"].reshape(BATCH*LENGTH,)
        
        loss = self.loss_func(hs, targets)
        return loss
    
    @torch.no_grad()
    def predict(self, max_seq_len=2400, device=None):
        tokenizer = self.tokenizer
        special_tokens = tokenizer.special_tokens
        
        tokens = tokenizer.get_pred_start()
        tokens = {k: v.to(device) for k, v in tokens.items()}
        preds = []
        pred_idx = 0
        
        while (pred_idx <= max_seq_len-1)\
        and ((len(preds) == 0) or (preds[-1] != special_tokens["eos"]-len(special_tokens))):
            
            if pred_idx >= 1:
                tokens = tokenizer.tokenize([torch.stack(preds)])
                tokens["value_tokens"][:, pred_idx+1] = special_tokens["pad"]
                tokens["padding_mask"][:, pred_idx+1] = True
            
            try:
                hs = self.forward(tokens, device=device)
            except IndexError:
                print(pred_idx)
                for k, v in tokens.items():
                    print(k)
                    print(v.shape)
                    print(v)
                import traceback
                print(traceback.format_exc())
                break

            hs = F.linear(hs[:, pred_idx], self.embedding.value_embed.weight)
            pred = hs.argmax(dim=1) - len(special_tokens)
            preds.append(pred[0])
            pred_idx += 1
            
        return torch.stack(preds) + len(special_tokens)

In [125]:
config = VertexPolyGenConfig(embed_dim=64, reformer__depth=1)
model = VertexPolyGen(config)

In [126]:
model.predict(max_seq_len=2400)

RuntimeError: Trying to create tensor with negative dimension -1: [-1]

In [ ]:
model(v_batch)

In [61]:
dec_tokenizer.tokenize([torch.tensor([0])])

{'value_tokens': tensor([[0, 3, 1, 2]]),
 'target_tokens': tensor([[3, 1, 2, 2]]),
 'coord_type_tokens': tensor([[0, 1, 0, 0]]),
 'position_tokens': tensor([[0, 1, 0, 0]]),
 'padding_mask': tensor([[False, False, False,  True]])}